In [2]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


HW 3.1

In [13]:
gold = pd.read_csv('gold.txt', dtype = str, delimiter = "\t", header = None)
gold = pd.DataFrame(gold.values, columns = ["url", "category"])

In [18]:
labels = pd.read_csv('labels.txt', dtype = str, delimiter = "\t", header = None)
labels = pd.DataFrame(labels.values, columns = ["turk","url","category"])

HW 3.2

In [31]:
## All urls in gold
labels_on_gold = labels.merge(gold, left_on = 'url', right_on = 'url', how= 'inner', 
                              suffixes = ['','_gold'])[['turk','url','category']]

labels_on_gold

,turk,url,category
0,A1253FXHCZ9CWM,http://0800-horoscope.com,G
1,A153PKAL7OAY36,http://0800-horoscope.com,G
2,A1FV9SAPL5C6KY,http://0800-horoscope.com,G
3,A1JTOT0DWM6QGL,http://0800-horoscope.com,G
4,A1PXXEOGQ76RNJ,http://0800-horoscope.com,G
...,...,...,...
3319,A35CBDDGBM7T6E,http://yousendit.com,G
3320,A3J86MK3VIE6ST,http://yousendit.com,G
3321,A4GP97LAQCCHJ,http://yousendit.com,G
3322,AAQNUEUU7H1Y5,http://yousendit.com,G


In [41]:
# urls not present in gold
labels_unknown = labels.merge(gold, left_on = 'url', right_on = 'url', how= 'left', 
                              suffixes = ['','_gold'])
labels_unknown = labels_unknown[labels_unknown['category_gold'].isnull()][['turk','url','category']]

HW 3.3

In [43]:
rater = labels.merge(gold, left_on = 'url', right_on = 'url', how= 'inner', 
                              suffixes = ['','_gold'])[['turk','url','category', 'category_gold']]

In [62]:
for i in range(len(rater)):
    if rater['category'][i] == rater['category_gold'][i]:
        rater['match'][i] = 1

<ipython-input-62-07e56005460e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rater['match'][i] = 1


In [122]:

rater_goodness = rater[['turk','match']]
rater_goodness = rater_goodness.groupby(['turk'])[['match']].agg(['count', 'sum'])['match']
rater_goodness['average_correctness'] = rater_goodness['sum']/rater_goodness['count']

In [123]:
rater_goodnessfinal = rater_goodness[['count','average_correctness']]
rater_goodnessfinal

,count,average_correctness
turk,,
A112DVP1KG4QZU,1,1.000000
A1253FXHCZ9CWM,29,0.517241
A12CY1Q7XKJJDE,1,1.000000
A12RE8G66WTO8B,20,0.750000
A12Y1GTGIQDGRA,3,0.333333
...,...,...
AYWOBMTEGFVVI,1,0.000000
AZ1S5BIG5R6P6,35,0.714286
AZSDW7JJL5VA7,100,0.680000


HW 3.4

In [127]:
rater_goodnessfinal['odds'] = (rater_goodnessfinal['average_correctness']) / (1.001 -  rater_goodnessfinal['average_correctness'] )
rater_goodnessfinal

HW 3.5

In [132]:
rater_goodnessfinal[rater_goodnessfinal['count'] > 20].sort_values('average_correctness', ascending = False)[:10]

,count,average_correctness,odds
turk,,,
A22C0PJUBFJTI0,36,0.916667,10.869565
A23YQUBXZPKILZ,24,0.875000,6.944444
ATVALOQVDCMZW,103,0.854369,5.826657
A1HIXWH4OXT8S4,40,0.825000,4.687500
A3220HG1O83HQ4,22,0.818182,4.475385
A32W20KGQXS0LL,25,0.800000,3.980100
AJSJVK40F5HM6,28,0.785714,3.649635
A31OCN4MNHUQ6W,184,0.777174,3.472222
A3MA9V9UPGCOK7,71,0.774648,3.422313


HW 3.6